In [247]:
import requests
import time
from bs4 import BeautifulSoup
from os import walk
from os.path import join
import random
import json
import warnings
import pandas as pd
from urllib.request import urlretrieve
from time import sleep


user_agents = [
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko)",
    "Mozilla/5.0 (compatible; MSIE 10.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11)',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)",
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.8',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.2; SV1; .NET CLR 1.1.4322)',
    'Mozilla/5.0(Macintosh;U;IntelMacOSX10_6_8;en-us)AppleWebKit/534.50',
    'Mozilla/5.0(Windows;U;WindowsNT6.1;en-us)AppleWebKit/534.50',
    'Opera/9.80(Macintosh;IntelMacOSX10.6.8;U;en)Presto/2.8.131Version/11.11',
    'Opera/9.80(WindowsNT6.1;U;en)Presto/2.8.131Version/11.11',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
]

user = random.choice(user_agents)
# print(user)
warnings.filterwarnings('ignore')
df=pd.DataFrame(columns=['商品型號','商品網址','商品照片','商品標題','品牌','分類','男女款','價格(台幣)',
                         '商品介紹','主要顏色','次要顏色','主色比例','次色比例'])
headers = { 'User-Agent': user ,
            'Accept': 'application/json, text/javascript, */*; q=0.01','Origin': 'https://www.skechers.com',
            'Referer': 'https://www.skechers.com/en-us/women/all?category=%2Fshoes%2Fcasuals&genders=W','Sec-Fetch-Mode': 'cors'
}

# 下一份內容網址的找法
url = 'https://www.skechers.com/en-us/api/html/products/styles/listing?category=%2Fshoes%2Fcasuals&genders=W&bookmark=g1AAAALzeJzLYWBg4MhgTmEwTs7JL01JzCtxyCzQNTQ30jUy1DXRNTI20yst1i1PLS7RNdRLzs8tKC1J1cvMK0ktykvMyQFqZkpkSJL___9_Fpjj5rD0DgMIJDEwsF7NosjgJAWQOfZIZi_TgZvtjc9sI0OCZjuAzIlHdvcluNmfcZptqmtkYkzQ7ASQOfXIZl-Dmc0lRpm781hABjUAKaDx8-HmX4C7nZsa5i-AmL8fbv4BuPlbKQsbiPkHIObfh5t_C26-JjXc_wBiPiL8d8DNL83KAgBH2Nsr'

while True:
    
    response = requests.get(url, headers=headers)


    # 找到下一份網址編碼

    html = response.json()["bookmark"]
    
#     print(html)
    # 創造下一份內容的網址
    url = 'https://www.skechers.com/en-us/api/html/products/styles/listing?category=%2Fshoes%2Fcasuals&genders=W&bookmark=' + html
#     time = random.randint(3, 5)
#     print("等待時間", time)
#     sleep(time)
    if response.json()['stylesHtml'] != '\n    \n                                \n\n    \n    \n    \n    \n\n    \n\n    \n        \n\n       \n\n            \n        \n            <div class=\"filters-no-results\">\n                <h1>No results found</h1>\n                <h2><a class=\"filters-clear-all\">clear all filters</a> to try again</h2>\n            </div>\n            \n        \n    ':
#         print(url)
#         進入網址後 對其內容作解析
        response = requests.get(url, headers=headers)
        html1 = response.json()['stylesHtml']
        shoes=BeautifulSoup(html1).find_all('div', class_="product-component js-collectionname-wrapper")
#         print(shoes)
        shoes = shoes[:300]
        # print(shoes)
        # ======
        url_list=[]
        img_list = []
        idx_list = []
        title_list = []
        price_list=[]
        context_list=[]
        for s in shoes:
        #     print(s)
            shoes_title=s.find('p',class_='product-name').text
            shoes_url ="https://www.skechers.com"+ s.find("a")["href"]
            shoes_price = s.find('p',class_='price').text.replace('$',"").replace('.00','')
        #     print(shoes_title,shoes_url,shoes_price)
            url_list.append(shoes_url)
            title_list.append(shoes_title)
            price_list.append(shoes_price)
        #     time = random.randint(1, 3)
        # #     print("等待時間", time)
        #     sleep(time)

        #     print(shoes_url)
        #     for i in url_list:
            response1= requests.get(shoes_url, headers = headers).text
            html1= BeautifulSoup(response1)
            shoes_context=html1.find('div',class_='toggle-expand toggle-description').text.strip()
            shoes_idx =html1.find('span',class_="pull-right product-label").text.replace('Style #: ',"").replace("  ","_")

            shoes_pic2 ='https://image.skechers.com/img/productimages/xlarge/'+ shoes_idx +'_'+ "E"+ '.jpg'

            shoes_pic1 ='https://image.skechers.com/img/productimages/xlarge/'+ shoes_idx +'_'+ "F"+ '.jpg'
            response1 = requests.get(shoes_pic1, headers = headers).text
            html1 = BeautifulSoup(response1)

        #     print(html1)
            try:
                if json.loads(str(html1))['error'] == 2000:
        #             print(shoes_pic2)
                    img_list.append(shoes_pic2)
            except:
        #          print(shoes_pic1)
                 img_list.append(shoes_pic1)   
        #     print(shoes_pic2)
        #     print(shoes_idx)
        #     print(shoes_context)
            idx_list.append(shoes_idx)
            context_list.append(shoes_context)

            time = random.randint(1, 3)
            print("等待時間", time)
            sleep(time)
        a=0
        while True:
            try:
                data = [idx_list[a],url_list[a],img_list[a],title_list[a],price_list[a],context_list[a]]
            except:
                break
            s = pd.Series(data,index=['商品型號','商品網址','商品照片','商品標題','價格(台幣)','商品介紹'])
            print(s)
            df =df.append(s,ignore_index=True)
            df.to_csv('E:/python/PyETL/skechers/woman/CASUALS.csv',index=False,encoding="utf-8-sig")        
            a +=1        
    
    else:
        break


# 進入網址後 對其內容作解析
# response = requests.get(url, headers=headers)
# html = response.json()['stylesHtml']
# shoes=BeautifulSoup(html).find_all('div', class_="product-component js-collectionname-wrapper")
# # print(shoes)

等待時間 2
等待時間 1
等待時間 3
等待時間 3
等待時間 3
等待時間 2
等待時間 1
等待時間 1
等待時間 2
等待時間 2
等待時間 2
等待時間 3
等待時間 1
等待時間 3
等待時間 3
等待時間 2
等待時間 1
等待時間 3
等待時間 2
等待時間 3
等待時間 2
等待時間 1
等待時間 3
等待時間 3
等待時間 2
等待時間 3
等待時間 1
等待時間 1
等待時間 3
等待時間 1
等待時間 2
等待時間 1
等待時間 1
等待時間 3
等待時間 1
等待時間 1
等待時間 2
等待時間 3
等待時間 2
等待時間 2
商品型號                                              33494_LPD
商品網址      https://www.skechers.com/en-us/style/33494/bob...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                        Women's BOBS B Cute - Meow Town
價格(台幣)                                                   42
商品介紹      Find the purr-fect feline print and soft weara...
dtype: object
商品型號                                              33496_GRY
商品網址      https://www.skechers.com/en-us/style/33496/bob...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                       Women's BOBS B Cute - Track Meet
價格(台幣)                                                   42
商品介紹      Get the most smiles per mile in soft

商品型號                                              33362_MLT
商品網址      https://www.skechers.com/en-us/style/33362/bob...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                    Women's BOBS Too Cozy - Classic Cat
價格(台幣)                                                   40
商品介紹      Snuggle up with your cats in comfort and timel...
dtype: object
商品型號                                              31248_LPD
商品網址      https://www.skechers.com/en-us/style/31248/bob...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                Women's BOBS Keepsakes 2.0 - Cuddle Cat
價格(台幣)                                                   50
商品介紹      Feel warm and comfy like a kitten sleeping in ...
dtype: object
商品型號                                             31242_CAMO
商品網址      https://www.skechers.com/en-us/style/31242/bob...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題            Women's BOBS Keepsakes 2.0 - Captain Cuddle
價格(台幣)      

dtype: object
商品型號                                             45703_CSNT
商品網址      https://www.skechers.com/en-us/style/45703/shi...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                            Women's Shindigs - Fortress
價格(台幣)                                                   55
商品介紹      Snuggle in style with the SKECHERS Shindigs-Fo...
dtype: object
商品型號                                             45703_CHSD
商品網址      https://www.skechers.com/en-us/style/45703/shi...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                            Women's Shindigs - Fortress
價格(台幣)                                                   55
商品介紹      Snuggle in style with the SKECHERS Shindigs-Fo...
dtype: object
商品型號                                             44926_CSNT
商品網址      https://www.skechers.com/en-us/style/44926/coz...
商品照片      https://image.skechers.com/img/productimages/x...
商品標題                  Women's Cozy Campfire - Bunny Cuddle